Requirements:

    Matlab
    Matlab's Image Processing Toolbox
    Matlab's Optimization Toolbox

Clone Airlocalize from its github repo:  
EITHER: in terminal (in your selected dir), type:  
`git clone git@github.com:timotheelionnet/AIRLOCALIZE.git`  
OR: Follow like its done in this notebook and clone it into the analysis dir.

In [ ]:
from glob import glob
import os

import sys
import os
import numpy as np
import csv
import copy
from scipy import spatial

import pandas as pd


In [ ]:
## analysis type: '' (for the original analysis) OR 'adj_spots'
analysis = '_adj_spots'
analysis = ''

### Define Dirs

In [ ]:
path = '/YOUR/PATH/'
org_ims_dir = os.path.join(path, f'simulations{analysis}')
al_dir = os.path.join(path, 'airlocalize')
os.makedirs(al_dir, exist_ok=True)

In [ ]:
org_ims_dir

In [ ]:
org_ims_dirs = glob(os.path.join(org_ims_dir, '*'))

### Clone Airlocalize Repo 

In [ ]:
wd = os.getcwd()
print(wd)
os.chdir(al_dir)
print(os.getcwd())

In [ ]:
!git clone git@github.com:timotheelionnet/AIRLOCALIZE.git 

In [ ]:
os.chdir(wd)
print(os.getcwd())

### Read example config file

In [ ]:
org_conf_file = open(os.path.join(al_dir, 'AIRLOCALIZE', 'examples/3DsmFISH_humCells.ini'), 'r')
lines = org_conf_file.readlines()
org_conf_file.close()

### Get lines to change

In [ ]:
lines[2], lines[5], lines[13], lines[18], lines[23], lines[37], lines[48], lines[51], lines[75], lines[101]

In [ ]:
d = {   ### constant:
    "fileProcessingMode": 13,
    "inclusionString": 18,
    "exclusionString": 23,
    "maxSpots": 51,
    "outputSpotsImage": 75,
    ## dirs:
    "dataFileName": 2,
    "saveDirName": 5,
    "psfSigma": 37,
    ## grid search
    "threshLevel": 48,
    "minDistBetweenSpots": 101} 

In [ ]:
# Set constant line values

lines[d["fileProcessingMode"]] = 'fileProcessingMode=batch\n'
lines[d["inclusionString"]] = 'inclusionString=.tif\n'
lines[d["exclusionString"]] = 'exclusionString=3000spots\n'
lines[d["maxSpots"]] = 'maxSpots=400\n'
lines[d["outputSpotsImage"]] = 'outputSpotsImage=0\n'

In [ ]:
# Values to set grid search:

thrs = [2,3,4,5,6,7,8,9,10,11,12,13]
dists = [0.2,1,2,3]


In [ ]:
conf_path_simul = os.path.join(al_dir, f'config_files_simulated{analysis}')
os.makedirs(conf_path_simul)

In [ ]:
results_path_simul = os.path.join(al_dir, f'results_simulated{analysis}')
os.makedirs(results_path_simul)

### Create Files

In [ ]:

for i,p in enumerate(org_ims_dirs):

    lines[d["dataFileName"]] = f'dataFileName={p}\n'

    # For origina analysis
    if analysis == '':
        sig_xy = p.split('Sigxy ')[1].split(' SigZ')[0]
        sig_xy = sig_xy.replace('pt','.') if len(sig_xy)>1 else sig_xy
        sig_z = p.split('SigZ ')[1]
    # For adj spots analysis
    else:
        sig_xy = p.split('sxy')[1].split('_sz')[0]
        sig_z =  p.split('_sz')[1].split('_')[0]

    lines[d["psfSigma"]] = f'psfSigma={sig_xy},{sig_z}\n'

    for thr in thrs:
        for dist in dists:

            lines[d["saveDirName"]] = f'saveDirName={results_path_simul}/{os.path.basename(p)}/{thr}_{dist}\n'

            lines[d["threshLevel"]] = f'threshLevel={thr}\n'
            lines[d["minDistBetweenSpots"]] = f'minDistBetweenSpots={dist}\n'

            out_file = open(os.path.join(conf_path_simul,f'{os.path.basename(p)}__{thr}_{dist}.ini'), "w")
            out_file.writelines(lines)
            out_file.close()


### Run on Matlab:

run matlab (can also run matlab --nodesktop) from the config file directory.  
in matlab :  

``` 
addpath(genpath(PATH/TO/AIRLOCALIZE));

conf_files = dir;

files = strings;

for c = 3:size(conf_files)   
    nam = conf_files(c).name   
    startt = tic    
    AIRLOCALIZE(nam);   
    files(c-2) = nam;    
    times(c-2) = toc(startt);    
end

T = table(files, times);  
writetable(T,'timetable.txt');
```